# Download the data (run only once)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nadyinky/sephora-products-and-skincare-reviews")
print("Path to dataset files:", path)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/raquel/.cache/kagglehub/datasets/nadyinky/sephora-products-and-skincare-reviews/versions/2


#### Imports

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load and clean the dataset

In [2]:
df = pd.read_csv("data/product_info.csv")
primary_category = set(df['primary_category'])
print(primary_category)

# Filter the DataFrame for rows where 'primary_category' is 'Skincare'
skincare_df = df[df['primary_category'] == 'Skincare']
print("Skin care data size:", len(skincare_df))

# Remove rows where highlight are non existent
skincare_df = skincare_df[skincare_df['highlights'].notna() & (skincare_df['highlights'] != '')]
print("Skin care data size after removing empty highlights:",len(skincare_df))

# Remove rows where ingredients are non existent
skincare_df = skincare_df[skincare_df['ingredients'].notna() & (skincare_df['ingredients'] != '')]
print("Skin care data size after removing empty ingredients",len(skincare_df))

# Remove where the secondary_category is empty
skincare_df = skincare_df[skincare_df['secondary_category'].notna() & (skincare_df['secondary_category'] != '')]
print("Skin care data size after removing empty secondary_category",len(skincare_df))

# # Remove where the tertiary_category is empty - no need
# skincare_df = skincare_df[skincare_df['tertiary_category'].notna() & (skincare_df['tertiary_category'] != '')]
# print("Skin care data size after removing empty tertiary_category",len(skincare_df))

{'Makeup', 'Bath & Body', 'Mini Size', 'Fragrance', 'Hair', 'Skincare', 'Men', 'Tools & Brushes', 'Gifts'}
Skin care data size 2420
Skin care data size after removing empty highlights 2003
Skin care data size after removing empty ingredients 1927
Skin care data size after removing empty secondary_category 1927


## Clean highlights column

In [3]:
highlights = skincare_df['highlights']
highlights = [h.replace("[", "").replace("]", "").replace("'", "").replace("Best for ", "").replace("Good for: ", "").replace(" Skin", "").replace("/", ", ") for h in highlights]
skincare_df['highlights'] = highlights

## Save cleaned dataset

In [4]:
skincare_df.to_csv('data/skincare.csv', index=False)

In [5]:
product_ids = skincare_df['product_id']
#print(len(df))
#print(len(df_review1))
#print("Different users:", len(set(df['author_id'])))

/var/folders/42/jzdlm0tx6bld5l7c53yw5zjc0000gn/T/ipykernel_22584/2568968107.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/reviews_0-250.csv")


602130
545756
Different users,  383697


In [6]:
# Load the CSV data into a DataFrame
df = pd.read_csv("data/reviews_0-250.csv")
df_review1 = df[df['product_id'].isin(product_ids)]

df_2 = pd.read_csv("data/reviews_250-500.csv")
df_review2 = df_2[df_2['product_id'].isin(product_ids)]

df_3 = pd.read_csv("data/reviews_500-750.csv")
df_review3 = df_3[df_3['product_id'].isin(product_ids)]

df_4 = pd.read_csv("data/reviews_750-1250.csv")
df_review4 = df_4[df_4['product_id'].isin(product_ids)]

df_5 = pd.read_csv("data/reviews_1250-end.csv")
df_review5 = df_5[df_5['product_id'].isin(product_ids)]

combined_df = pd.concat([df_review2, df_review1], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review3], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review4], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review5], axis=0, ignore_index=True)

#print(len(combined_df))
#print(len(set(combined_df['author_id'])))

/var/folders/42/jzdlm0tx6bld5l7c53yw5zjc0000gn/T/ipykernel_22584/1564603506.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_4 = pd.read_csv("data/reviews_750-1250.csv")
/var/folders/42/jzdlm0tx6bld5l7c53yw5zjc0000gn/T/ipykernel_22584/1564603506.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_5 = pd.read_csv("data/reviews_1250-end.csv")


966018
513998


### Excluding products with "Mini" or "Limited Edition" in names

In [7]:
# Be mindful that "mini" and "Mini" is the same when removing in the below
mini_in_product_name = combined_df['product_name'].str.contains('mini', case=False, na=False)
limited_edition_in_product_name = combined_df['product_name'].str.contains('limited edition', case=False, na=False)

rows_to_exclude = mini_in_product_name | limited_edition_in_product_name

cleaned_df = combined_df[~rows_to_exclude]

### Users with at least 10 reviews

In [18]:
filtered_df = cleaned_df.groupby('author_id').filter(lambda x: x['product_id'].nunique() > 9)

# Display the filtered DataFrame
print(f"Number of rows in the filtered dataframe: {len(filtered_df)}")
print(f"Number of unique authors in the filtered dataframe: {len(set(filtered_df['author_id']))}")
#set(filtered_df['author_id'])

88723
4470


In [19]:
# Combining all the the products reviewed for each person in the dataset
selected_columns = filtered_df[['author_id', 'product_id']]
combined_reviews = selected_columns.groupby('author_id')['product_id'].apply(lambda x: ' '.join(x)).reset_index()
combined_reviews

,author_id,product_id
0,11777122,P232915 P466123 P421277 P500777 P475124 P48232...
1,967124371,P7880 P430337 P4016 P466123 P421277 P405096 P4...
2,989697609,P456218 P500777 P480461 P504045 P482325 P50381...
3,1048276344,P430337 P302103 P455676 P421243 P421998 P44595...
4,1148829653,P232915 P421235 P466123 P472469 P480461 P50404...
...,...,...
4465,998162812,P422022 P455611 P379707 P269122 P309308 P42995...
4466,998179876,P479841 P455236 P501760 P501254 P466153 P47715...
4467,998853649,P456211 P470041 P447212 P447782 P447781 P45408...
4468,9990263118,P456566 P467655 P440307 P410400 P432668 P43454...


In [20]:
# Convert the 'reviews' into a list of transactions
transactions = combined_reviews['product_id'].str.split().tolist()

# Create a DataFrame for one-hot encoding
# Flatten all unique items (reviews) and create a unique item list
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# Apply the Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

# Focus on only pairs of reviews
frequent_pairs = frequent_itemsets[frequent_itemsets['itemsets'].apply(len) == 2]

# Generate association rules, with support as minimum of 5%
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.05, num_itemsets=len(frequent_itemsets))

print(frequent_pairs)

      support            itemsets
85   0.056823  (P309308, P270594)
86   0.077405  (P400259, P270594)
87   0.063758  (P411365, P270594)
88   0.062864  (P423148, P270594)
89   0.091946  (P423688, P270594)
..        ...                 ...
196  0.058837  (P503936, P503726)
197  0.051007  (P505031, P503936)
198  0.052796  (P505023, P505009)
199  0.053468  (P505031, P505023)
200  0.052125  (P505054, P505023)

[116 rows x 2 columns]


In [21]:
rules_display = rules[["antecedents", "consequents", "support", "confidence", "lift"]].copy()

# Convert frozensets to readable strings
rules_display["antecedents"] = rules_display["antecedents"].apply(lambda x: ', '.join(list(x)))
rules_display["consequents"] = rules_display["consequents"].apply(lambda x: ', '.join(list(x)))

# Sorting by confidence
rules_display = rules_display.sort_values(by="confidence", ascending=False)
print(rules_display.to_string(index=False))

     antecedents      consequents  support  confidence      lift
         P482551          P483076 0.094855    1.000000 10.542453
P500633, P482551          P483076 0.059284    1.000000 10.542453
         P483076          P482551 0.094855    1.000000 10.542453
P483076, P270594          P482551 0.057047    1.000000 10.542453
P482551, P270594          P483076 0.057047    1.000000 10.542453
P500633, P483076          P482551 0.059284    1.000000 10.542453
P501254, P270594          P500633 0.052349    0.780000  2.795990
P501760, P270594          P500633 0.054810    0.760870  2.727415
         P505054          P505023 0.052125    0.749196  7.935798
P482676, P270594          P500633 0.055928    0.726744  2.605089
P503936, P479841          P500633 0.052573    0.723077  2.591944
         P500288          P500633 0.059732    0.719677  2.579755
         P501254          P500633 0.081655    0.718504  2.575551
P479841, P270594          P500633 0.069575    0.695749  2.493986
         P471043         

In [22]:
# let's try to quantify this
unique_values = pd.concat([rules_display['antecedents'], rules_display['consequents']]).nunique()
unique_values

68

## Testing

In [27]:
filtered_rules = rules_display[rules_display['antecedents'].apply(lambda x: 'P447596' in x)]
print(filtered_rules)

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [28]:
filtered_rules = rules_display[rules_display['antecedents'].apply(lambda x: 'P500633' in x)]
print(filtered_rules)

          antecedents       consequents   support  confidence       lift
335  P500633, P482551           P483076  0.059284    1.000000  10.542453
334  P500633, P483076           P482551  0.059284    1.000000  10.542453
232  P500633, P400259           P270594  0.050783    0.656069   2.476883
304  P500633, P503936           P270594  0.083893    0.647668   2.445167
298  P501760, P500633           P270594  0.054810    0.644737   2.434099
..                ...               ...       ...         ...        ...
204           P500633           P503668  0.051454    0.184443   2.198557
184           P500633           P482540  0.051454    0.184443   2.389735
210           P500633           P505009  0.051230    0.183641   2.121122
235           P500633  P400259, P270594  0.050783    0.182037   2.351748
178           P500633           P481831  0.050336    0.180433   1.804330

[76 rows x 5 columns]
